## Problema classificación estadio de Cirrosis AIDE

In [5]:
import AIDE as aide
import openai
import pandas as pd

El modelo a utilizar es gpt-4-turbo

In [4]:
pd.read_csv('Data/liver_cirrhosis.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         25000 non-null  int64  
 1   Status         25000 non-null  object 
 2   Drug           25000 non-null  object 
 3   Age            25000 non-null  int64  
 4   Sex            25000 non-null  object 
 5   Ascites        25000 non-null  object 
 6   Hepatomegaly   25000 non-null  object 
 7   Spiders        25000 non-null  object 
 8   Edema          25000 non-null  object 
 9   Bilirubin      25000 non-null  float64
 10  Cholesterol    25000 non-null  float64
 11  Albumin        25000 non-null  float64
 12  Copper         25000 non-null  float64
 13  Alk_Phos       25000 non-null  float64
 14  SGOT           25000 non-null  float64
 15  Tryglicerides  25000 non-null  float64
 16  Platelets      25000 non-null  float64
 17  Prothrombin    25000 non-null  float64
 18  Stage 

In [9]:
data_dir = "Data/"
goal="Make a classifier between the 3 values in Stage columns, split the dataset into .7 train, .15 validation and .15 testing"
eval="Use a report, the accuracy and f1-score"

In [10]:
exp = aide.Experiment(
    data_dir=data_dir,
    eval=eval,
    goal=goal
)

In [11]:
response = exp.run(steps=10)

/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-aide-venv/lib/python3.10/site-packages/AIDE/utils/tree_export.py:34: RuntimeWarning: invalid value encountered in divide
  layout = (layout - layout.min(axis=0)) / (layout.max(axis=0) - layout.min(axis=0))


In [15]:
print(f'Validación de la solución: {response.valid_metric}')

Validación de la solución: 0.9547


In [16]:
print(f'El código proporcionado es : \n\n{response.code}')

El código proporcionado es : 

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load data
data = pd.read_csv("./input/liver_cirrhosis.csv")

# Encode categorical variables
label_encoders = {}
categorical_columns = data.select_dtypes(include=["object"]).columns
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split data into features and target
X = data.drop("Stage", axis=1)
y = data["Stage"]

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Parameter grid for RandomForest
pa

Tiempo de ejecución --> <label style="color:red"> > 59min </label>

Coste de ejecución --> <label style="color:red">$0.45 </label>

Ahora ejecutamos el código proporcionado para comprobar la solución de AIDE

In [17]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load data
data = pd.read_csv("./Data/liver_cirrhosis.csv")

# Encode categorical variables
label_encoders = {}
categorical_columns = data.select_dtypes(include=["object"]).columns
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split data into features and target
X = data.drop("Stage", axis=1)
y = data["Stage"]

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Parameter grid for RandomForest
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
}

# Initialize and train the Random Forest classifier with GridSearchCV
rf_classifier = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf_classifier, param_grid=param_grid, cv=5, scoring="accuracy"
)
grid_search.fit(X_train, y_train)

# Best estimator
best_rf = grid_search.best_estimator_

# Predict on validation set
y_val_pred = best_rf.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
f1_scores = classification_report(y_val, y_val_pred, output_dict=True)["weighted avg"][
    "f1-score"
]

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1-Score: {f1_scores:.4f}")

# Prepare submission of test predictions
y_test_pred = best_rf.predict(X_test)
submission = pd.DataFrame({"Stage": y_test_pred})
submission.to_csv("./Data/submission.csv", index=False)


Validation Accuracy: 0.9547
Validation F1-Score: 0.9547


In [18]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           1       0.96      0.94      0.95      1240
           2       0.94      0.95      0.95      1266
           3       0.96      0.97      0.97      1244

    accuracy                           0.95      3750
   macro avg       0.95      0.95      0.95      3750
weighted avg       0.95      0.95      0.95      3750

